# CAI Lab Session 4: Implementing search in the vector space model

In this session you will:

- Continue to work with the `arxiv` repository from last session
- Learn how to do atomic, conjunctive and disjunctive search with ElasticSearch
- Build an inverted index for the `arxiv` repository from last session (should fit in main memory)
- Implement search in the vector space model and compare it with ElasticSearch built-in search mechanism
- Compare different implementations of search

## 1. Built-in search in ElasticSearch

ElasticSearch provides a search mechanism to make queries against a database. 
In the next code snippet you can find examples on how to do this with an atomic query (single term)
and with conjunctive and disjunctive queries.

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl.query import Q


client = Elasticsearch("http://localhost:9200", request_timeout=1000)
s = Search(using=client, index='arxiv')

## atomic query
q = Q('query_string',query='computer')  # Feel free to change the word

s = s.query(q)  # add the query to the search object
response = s[:5].execute()  # execute the search and return the first 5 results
for r in response:  # only returns a specific number of results
    print('ID= %s SCORE=%s' % (r.meta.id,  r.meta.score))
    print('PATH= %s' % r.path)
    print('TEXT: %s' % r.text[:90])
    print()

ID= -NyUMosBHh_bflI4QOFP SCORE=3.2082987
PATH= ../../../arxiv_abs/arxiv\cs.updates.on.arXiv.org/002772
TEXT: Limit computable functions can be characterized by Turing jumps on the input side or limit

ID= 0N2UMosBHh_bflI4cDBZ SCORE=3.2082987
PATH= ../../../arxiv_abs/arxiv\math.updates.on.arXiv.org/001904
TEXT: Limit computable functions can be characterized by Turing jumps on the input side or limit

ID= eNyUMosBHh_bflI4TvNV SCORE=3.1941829
PATH= ../../../arxiv_abs/arxiv\cs.updates.on.arXiv.org/007252
TEXT: We study scheduling of computation tasks across $n$ workers in a large scale distributed l

ID= bN2UMosBHh_bflI4dDgC SCORE=3.1941829
PATH= ../../../arxiv_abs/arxiv\math.updates.on.arXiv.org/003852
TEXT: We study scheduling of computation tasks across $n$ workers in a large scale distributed l

ID= htyUMosBHh_bflI4T_TR SCORE=3.1520977
PATH= ../../../arxiv_abs/arxiv\cs.updates.on.arXiv.org/007522
TEXT: Grid Computing is an idea of a new kind of network technology in which research wor

In [2]:
## conjunctive query

client = Elasticsearch("http://localhost:9200", request_timeout=1000)
s = Search(using=client, index='arxiv')

q = Q('query_string',query='computer') & Q('query_string',query='magic')

s = s.query(q)
response = s[0:5].execute()
for r in response:  # only returns a specific number of results
    print(f'ID= {r.meta.id} SCORE={r.meta.score}')
    print(f'PATH= {r.path}')
    print(f'TEXT: {r.text[:90]}')
    print()

ID= 5t2UMosBHh_bflI4hWSJ SCORE=14.512409
PATH= ../../../arxiv_abs/arxiv\quant-ph.updates.on.arXiv.org/000650
TEXT: We give a new algorithm for computing the robustness of magic - a measure of the utility o

ID= Ad2UMosBHh_bflI4hWWJ SCORE=14.512409
PATH= ../../../arxiv_abs/arxiv\quant-ph.updates.on.arXiv.org/000677
TEXT: We give a new algorithm for computing the robustness of magic - a measure of the utility o

ID= 0N2UMosBHh_bflI4h2gT SCORE=11.029575
PATH= ../../../arxiv_abs/arxiv\quant-ph.updates.on.arXiv.org/001652
TEXT: A defining feature in the field of quantum computing is the potential of a quantum device 

ID= vNyUMosBHh_bflI4A55F SCORE=10.839215
PATH= ../../../arxiv_abs/arxiv\astro-ph.updates.on.arXiv.org/006224
TEXT: Context. PKS 1510-089 is a flat spectrum radio quasar strongly variable in the optical and

ID= etyUMosBHh_bflI4BaEv SCORE=9.807085
PATH= ../../../arxiv_abs/arxiv\astro-ph.updates.on.arXiv.org/006926
TEXT: PKS 1510-089 is a flat spectrum radio quasar strongly vari

In [3]:
## disjunctive query

client = Elasticsearch("http://localhost:9200", request_timeout=1000)
s = Search(using=client, index='arxiv')

q = Q('query_string',query='computer') | Q('query_string',query='magic')

s = s.query(q)
response = s[0:5].execute()
for r in response:  # only returns a specific number of results
    print(f'ID= {r.meta.id} SCORE={r.meta.score}')
    print(f'PATH= {r.path}')
    print(f'TEXT: {r.text[:90]}')
    print()

ID= 5t2UMosBHh_bflI4hWSJ SCORE=14.512409
PATH= ../../../arxiv_abs/arxiv\quant-ph.updates.on.arXiv.org/000650
TEXT: We give a new algorithm for computing the robustness of magic - a measure of the utility o

ID= Ad2UMosBHh_bflI4hWWJ SCORE=14.512409
PATH= ../../../arxiv_abs/arxiv\quant-ph.updates.on.arXiv.org/000677
TEXT: We give a new algorithm for computing the robustness of magic - a measure of the utility o

ID= h9yUMosBHh_bflI4M89P SCORE=12.0883665
PATH= ../../../arxiv_abs/arxiv\cond-mat.updates.on.arXiv.org/003482
TEXT: When two monolayers of graphene are stacked with a small relative twist angle, the resulti

ID= aN2UMosBHh_bflI4ayQI SCORE=11.981175
PATH= ../../../arxiv_abs/arxiv\hep-th.updates.on.arXiv.org/000265
TEXT: We introduce the extended Freudenthal-Rosenfeld-Tits magic square based on six algebras: t

ID= G92UMosBHh_bflI4bi3m SCORE=11.981175
PATH= ../../../arxiv_abs/arxiv\math.updates.on.arXiv.org/000955
TEXT: We introduce the extended Freudenthal-Rosenfeld-Tits magic squ

## 2. Excruciatingly slow search

In class we have presented a _slow_ version of search that, given a search query $q$, loops over every document in the database
computing the cosine similarity between document and query. Once this is done, it sorts documents by their similarity w.r.t. $q$ and returns the top $r$
scoring ones. 

```
1. for each d in D:
    sim(d,q) = 0
    get vector representing d
    for each w in q:
        sim(d,q) += tf(d,w) * idf(w)
    normalize sim(d,q) by |d|*|q|
2. sort results by similarity
3. return top r docs
```

A possible implementation can be found below. 

__Remark:__ _It should be important to note that there are certain elements in the implementation below that refer to my own
implementation, and that you should adapt to your own; in particular, the line_

```    weights = dict(normalize(tf_idf(s['_id'])))   # gets weights as a python dict of term -> weight ```

_obtains tf-idf weights through calling a function `tf_idf` that I have implemented that, given a docid, returns a list of pairs (term, weight); and `normalize` takes such a list a normalizes weights so that the corresponding vector has length 1. 
Obviously, you should adapt the code to your own implementations from previous sessions._


In [4]:
from elasticsearch.helpers import scan
from pprint import pprint
from elasticsearch import Elasticsearch
import tqdm
import numpy as np  

In [5]:
# get tf-idf list of pairs (term, weight) from doc (internal) id
def tf_idf(idx: str, client, doc_id: str, D: int) -> list:
    """
    Compute tf-idf for each term in a document with internal id doc_id
    """

    import math

    tv = client.termvectors(index=idx, id=doc_id, fields=['text'], term_statistics=True)

    tfidf = []
    if 'text' in tv['term_vectors']:
        max_word = max(tv['term_vectors']['text']['terms'], key=lambda x: tv['term_vectors']['text']['terms'][x]['term_freq'])
        max_freq = tv['term_vectors']['text']['terms'][max_word]['term_freq']
        
        for t in tv['term_vectors']['text']['terms']:
            word = t
            
            word_freq = tv['term_vectors']['text']['terms'][t]['term_freq']
            df = tv['term_vectors']['text']['terms'][t]['doc_freq']

            tfidf.append((word, (word_freq/max_freq) * math.log(D/df, 2)))
    return tfidf

In [6]:
# normalizes weights so that resulting vec has length 1
def normalize(tdfidf: list) -> list:
    """
    Normalize tf-idf weights so that the resulting vector has length 1
    """

    import math

    norm_d = math.sqrt(sum([w**2 for _, w in tdfidf]))
    return [(t, w/norm_d) for t, w in tdfidf]

In [7]:

client = Elasticsearch("http://localhost:9200", request_timeout=1000)

r = 10  # only return r top docs
query = 'computer magic'
sims = dict()

l2query  = np.sqrt(len(query.split()))  # l2 of query assuming 0-1 vector representation

# get nr. of docs; just for the progress bar
ndocs = int(client.cat.count(index='arxiv', format = "json")[0]['count'])  # D

# scan through docs, compute cosine sim between query and each doc
for s in tqdm.tqdm(scan(client, index='arxiv', query={"query" : {"match_all": {}}}), total=ndocs):
    docid = s['_source']['path']   # use path as id

    sims[docid] = 0.0
    weights = dict(normalize(tf_idf('arxiv', client, s['_id'], ndocs)))  # normalize weights for doc

    for w in query.split():  # gets terms as a list
        if w in weights:    # probably need to do something fancier to make sure that word is in vocabulary etc.
            sims[docid] += weights[w]   # accumulates if w in current doc

    # normalize sim
    sims[docid] /= l2query  # ||q||_2 = 1

# now sort by cosine similarity
sorted_answer = sorted(sims.items(), key=lambda kv: kv[1], reverse=True)
pprint(sorted_answer[:r])

  0%|          | 1/58102 [00:00<2:04:32,  7.78it/s]

100%|██████████| 58102/58102 [10:26<00:00, 92.80it/s] 


[('../../../arxiv_abs/arxiv\\quant-ph.updates.on.arXiv.org/000650',
  0.47752597051076895),
 ('../../../arxiv_abs/arxiv\\quant-ph.updates.on.arXiv.org/000677',
  0.477263052236843),
 ('../../../arxiv_abs/arxiv\\cond-mat.updates.on.arXiv.org/003482',
  0.38792349570755896),
 ('../../../arxiv_abs/arxiv\\quant-ph.updates.on.arXiv.org/001475',
  0.3023262889426185),
 ('../../../arxiv_abs/arxiv\\hep-th.updates.on.arXiv.org/000265',
  0.26483163493536094),
 ('../../../arxiv_abs/arxiv\\math.updates.on.arXiv.org/000955',
  0.26483163493536094),
 ('../../../arxiv_abs/arxiv\\hep-th.updates.on.arXiv.org/000255',
  0.24832299488605178),
 ('../../../arxiv_abs/arxiv\\math.updates.on.arXiv.org/000896',
  0.24832299488605178),
 ('../../../arxiv_abs/arxiv\\astro-ph.updates.on.arXiv.org/002083',
  0.24062631526975578),
 ('../../../arxiv_abs/arxiv\\astro-ph.updates.on.arXiv.org/001135',
  0.23778079125705068)]


In [8]:
# from elasticsearch.helpers import scan
# from pprint import pprint
# from elasticsearch import Elasticsearch
# import tqdm
# import numpy as np

# # get tf-idf vector from doc (internal) id
# def tf_idf(doc_id):
#     # does nothing, adapt to your needs
#     return []

# # normalizes weights so that resulting vec has length 1
# def normalize(l1):
#     # does nothing, adapt to your needs
#     return l1

# client = Elasticsearch("http://localhost:9200", request_timeout=1000)

# r = 10  # only return r top docs
# query = 'computer magic'
# sims = dict()

# l2query  = np.sqrt(len(query.split()))  # l2 of query assuming 0-1 vector representation

# # get nr. of docs; just for the progress bar
# ndocs = int(client.cat.count(index='arxiv', format = "json")[0]['count'])

# # scan through docs, compute cosine sim between query and each doc
# for s in tqdm.tqdm(scan(client, index='arxiv', query={"query" : {"match_all": {}}}), total=ndocs):
#     docid = s['_source']['path']   # use path as id
    
#     sims[docid] = 0.0
#     weights = dict(normalize(tf_idf(s['_id'])))   # get tf-idf weights representing doc as dict
#     for w in query.split():  # gets terms as a list
#         if w in weights:    # probably need to do something fancier to make sure that word is in vocabulary etc.
#             sims[docid] += weights[w]   # accumulates if w in current doc
#     # normalize sim
#     sims[docid] /= l2query

# # now sort by cosine similarity
# sorted_answer = sorted(sims.items(), key=lambda kv: kv[1], reverse=True)
# pprint(sorted_answer[:r])


In [9]:
nz = len([x for x, s in sorted_answer if s>0])
total = len(sorted_answer)
print(f'There are {nz} docs with non-zero similarity out of {total}, i.e. {100.0*nz/total:.1f}%')

There are 140 docs with non-zero similarity out of 58102, i.e. 0.2%


## 3. Your tasks

---

**Exercise 1:**  

Make sure you understand the algorithm for implementing search described in the lecture notes. Both slow and efficient versions. Describe
the number of sums you need to do in both slow and quick versions for the following toy example with a vocabulary of size 4 and four documents:

- $q = 0,1,1,0$

- document-term matrix:
<center>


|        | t1  | t2  | t3  | t4  |
|--------|-----|-----|-----|-----|
| **d1** | 1.2 | 0.0 | 0.0 | 0.0 |
| **d2** | 0.7 | 0.3 | 1.5 | 0.1 |
| **d3** | 0.0 | 0.0 | 0.0 | 0.7 |
| **d4** | 2.0 | 0.0 | 0.0 | 0.0 |

</center>

---



**Exercise 2:**

Implement the quick version; run both slow and quick versions and report times (as a reference, in my old laptop it takes around 5m30s to run the slow version in the code above). Make sure both versions return the same answer. Note that you will need to build an inverted index in order to implement the efficient version as explained in class; it may take time but this is done once for all queries, and can be done "off-line".

---


In [1]:
posting_list = dict()

for s in tqdm.tqdm(scan(client, index='arxiv', query={"query" : {"match_all": {}}}), total=ndocs):
  docid = s['_id']
  
  tv = client.termvectors(index='arxiv', id=docid, fields=['text'], term_statistics=True)
  
  if 'text' in tv['term_vectors']:
    for t in tv['term_vectors']['text']['terms']:
      if t not in posting_list:
        posting_list[t] = set()
      posting_list[t].add(docid)
      
pprint(posting_list)

NameError: name 'tqdm' is not defined

In [12]:
def inverted_file_implementation(query: str,  
                                 client: Elasticsearch,
                                 posting_list: dict, 
                                 ndocs:int, 
                                 r: int) -> list:
  """
  Implement inverted file retrieval for a query and return top r results
  """
  sims = dict()
  
  for w in query.split(" "):
    L = posting_list[w]
    for d in L:
      weights = dict(normalize(tf_idf('arxiv', client, d, ndocs)))
      if d not in sims:
        sims[d] = 0.0 
      sims[d] += weights[w]
  
  l2query = np.sqrt(len(query.split()))  
  for d in sims:
    sims[d] /= l2query
    
  sorted_answer = sorted(sims.items(), key=lambda kv: kv[1], reverse=True)
  return sorted_answer[:r]

query = 'computer magic'
r = 10
idx = 'arxiv'
ndocs = int(client.cat.count(index='arxiv', format = "json")[0]['count'])

inverted_file_implementation(query, client, posting_list, ndocs, r)

[('5t2UMosBHh_bflI4hWSJ', 0.47752597051076895),
 ('Ad2UMosBHh_bflI4hWWJ', 0.477263052236843),
 ('h9yUMosBHh_bflI4M89P', 0.38792349570755896),
 ('H92UMosBHh_bflI4h2gT', 0.3023262889426185),
 ('aN2UMosBHh_bflI4ayQI', 0.26483163493536094),
 ('G92UMosBHh_bflI4bi3m', 0.26483163493536094),
 ('4N2UMosBHh_bflI4bizm', 0.24832299488605178),
 ('Xt2UMosBHh_bflI4ayQI', 0.24832299488605178),
 ('j9yTMosBHh_bflI48I7S', 0.24062631526975578),
 ('29yTMosBHh_bflI47Ioz', 0.23778079125705068)]


**Exercise 3:**

Compare the results for a few sample queries that you get from your quick version and ElasticSearch search. Do you get similar results? Which is faster?

---

## 4. Rules of delivery

- To be solved in _pairs_.

- No plagiarism; don't discuss your work with other teams. You can ask for help to others for simple things, such as recalling a python instruction or module, but nothing too specific to the session.

- If you feel you are spending much more time than the rest of the classmates, ask us for help. Questions can be asked either in person or by email, and you'll never be penalized by asking questions, no matter how stupid they look in retrospect.

- Write a short report listing the solutions to the exercises proposed. Include things like the important parts of your implementation (data structures used for representing objects, algorithms used, etc). You are welcome to add conclusions and findings that depart from what we asked you to do. We encourage you to discuss the difficulties you find; this lets us give you help and also improve the lab session for future editions.

- Turn the report to PDF. Make sure it has your names, date, and title. Include your code in your submission.

- Submit your work through the [raco](http://www.fib.upc.edu/en/serveis/raco.html) _before November 6th, 2023_.